In [2]:
import pandas as pd


In [3]:
#import all flat files

INF_Feature = pd.read_excel (r'/Users/joezhou/Downloads/C_ALL_INFO_FEATURES.xlsx') 
PRICE_Feature = pd.read_excel (r'/Users/joezhou/Downloads/C_ALL_Prices_features.xlsx')
FIN_Feature = pd.read_excel (r'/Users/joezhou/Downloads/C_ALL_FIN_features.xlsx')
DIV_Feature = pd.read_excel (r'/Users/joezhou/Downloads/C_ALL_Div_FEATURES.xlsx') 
 


In [4]:
# Find data coverage

import pandas as pd

TIC = pd.read_csv(r'/Users/joezhou/Downloads/C_ALL_TICKER.csv')
TIC_U = TIC.drop_duplicates(subset=['colummn']).rename(columns={'colummn': 'TickName'})[['TickName']]
TIC_U['Counter'] = 1


INF_U = INF_Feature.drop_duplicates(subset=['TickName'])[['TickName']]
INF_U['IN_INFO']="In Info Extract"

PRICE_U = PRICE_Feature.drop_duplicates(subset=['TickName'])[['TickName']]
PRICE_U['IN_PRICE']="In Price Extract"

FIN_U = FIN_Feature.drop_duplicates(subset=['TickName'])[['TickName']]
FIN_U['IN_FIN']="In Financial Extract"

DIV_U = DIV_Feature.drop_duplicates(subset=['TickName'])[['TickName']]
DIV_U['IN_DIV']="In Dividend Extract"


In [5]:
# Join it back together
TIC_COVERAGE = TIC_U.merge(INF_U,how='left',on='TickName').merge(PRICE_U,how='left',on='TickName').merge(FIN_U,how='left',on='TickName').merge(DIV_U,how='left',on='TickName').fillna('Missing')


Tbl_Coverage = TIC_COVERAGE.groupby(['IN_INFO','IN_PRICE','IN_FIN', 'IN_DIV'])['Counter'].count().reset_index().sort_values(by=['Counter'],ascending=False)
Tbl_Coverage['Percentage'] = 100 * Tbl_Coverage['Counter']  / Tbl_Coverage['Counter'].sum()

Tbl_Coverage.head(27)
# To-do: add market cap data in here to quantify the proportion missing

,IN_INFO,IN_PRICE,IN_FIN,IN_DIV,Counter,Percentage
1,In Info Extract,In Price Extract,In Financial Extract,Missing,918,42.777260
5,In Info Extract,Missing,In Financial Extract,Missing,444,20.689655
0,In Info Extract,In Price Extract,In Financial Extract,In Dividend Extract,264,12.301957
11,Missing,Missing,Missing,Missing,178,8.294501
4,In Info Extract,Missing,In Financial Extract,In Dividend Extract,125,5.824790
3,In Info Extract,In Price Extract,Missing,Missing,108,5.032619
7,In Info Extract,Missing,Missing,Missing,69,3.215284
2,In Info Extract,In Price Extract,Missing,In Dividend Extract,20,0.931966
6,In Info Extract,Missing,Missing,In Dividend Extract,9,0.419385
9,Missing,In Price Extract,Missing,Missing,5,0.232992


In [6]:

#Remove tickers without complete variable set
dp_cov_comp = TIC_COVERAGE[(TIC_COVERAGE.IN_INFO == 'In Info Extract') 
                    & (TIC_COVERAGE.IN_PRICE == 'In Price Extract')
                    & (TIC_COVERAGE.IN_FIN == 'In Financial Extract')
                    & (TIC_COVERAGE.IN_DIV == 'In Dividend Extract')
                    ]




In [7]:
dp_cov_comp.count()

TickName    264
Counter     264
IN_INFO     264
IN_PRICE    264
IN_FIN      264
IN_DIV      264
dtype: int64

In [8]:

# Combine the feature sets

TIC_FLAT = dp_cov_comp.merge(PRICE_Feature,how='left',on='TickName').merge(DIV_Feature,how='left',on='TickName').merge(INF_Feature,how='left',on='TickName')

# .merge(FIN_Feature,how='left',on='TickName')


# .fillna('Missing')

# merge(INF_Feature,how='left',on='TickName').


In [9]:
TIC_FLAT.head(5)

,TickName,Counter,IN_INFO,IN_PRICE,IN_FIN,IN_DIV,Unnamed: 0_x,absmvmtOPEN_MEDIAN,absmvmtOPEN_STD,Unnamed: 0_y,...,fullTimeEmployees,forwardEps,forwardPE,shortName,longName,longBusinessSummary,website,messageBoardId,industry,sector
0,ONT.AX,1,In Info Extract,In Price Extract,In Financial Extract,In Dividend Extract,1,0.007143,0.016702,0,...,337.0,0.36,19.277777,SMILES LTD FPO,1300SMILES Limited,1300SMILES Limited provides dental and managem...,http://www.1300smiles.com.au,finmb_21811671,Medical Care Facilities,Healthcare
1,TGP.AX,1,In Info Extract,In Price Extract,In Financial Extract,In Dividend Extract,7,0.024242,0.014032,2,...,NaN,0.05,18.699999,360 GROUP STAPLED,360 Capital Group Limited,"360 Capital Group is an ASX-listed, investment...",http://www.360capital.com.au,finmb_22986008,Real Estate Services,Real Estate
2,TOT.AX,1,In Info Extract,In Price Extract,In Financial Extract,In Dividend Extract,8,0.011364,0.006939,3,...,NaN,NaN,NaN,360CAPITAL STAPLED,360 Capital REIT,360 Capital REIT invests across the entire rea...,http://www.360capital.com.au/investment-strate...,finmb_289139858,REIT—Diversified,Real Estate
3,5GN.AX,1,In Info Extract,In Price Extract,In Financial Extract,In Dividend Extract,14,0.041177,0.026274,4,...,NaN,NaN,NaN,5G NETWORK FPO,5G Networks Limited,"5G Networks Limited, together with its subsidi...",http://www.5gnetworks.com.au,finmb_537651034,Telecom Services,Communication Services
4,ABP.AX,1,In Info Extract,In Price Extract,In Financial Extract,In Dividend Extract,25,0.024476,0.010031,6,...,NaN,0.27,10.074074,ABACUS STAPLED,Abacus Property Group,"Abacus Property Group (ASX:ABP), is a diversif...",http://www.abacusproperty.com.au,finmb_7657684,REIT—Diversified,Real Estate


In [10]:
TIC_FLAT.to_excel('/Users/joezhou/Downloads/C_TIC_FLAT.xlsx', engine='xlsxwriter')  

